![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_MPNetForQuestionAnswering.ipynb)

# Import ONNX MPNet models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in `Spark NLP 5.0.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- The MPNetForQuestionAnswering model was introduced in `Spark NLP 5.2.4`

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers[onnx]==4.48.3 optimum onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use the [shaina/covid_qa_mpnet](https://huggingface.co/shaina/covid_qa_mpnet) model from HuggingFace as an example and export it.
- In addition to the MPNet model, we also need to save the tokenizer. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import MPNetTokenizer
from optimum.onnxruntime import ORTModelForQuestionAnswering

MODEL_NAME = "haddadalwi/multi-qa-mpnet-base-dot-v1-finetuned-squad2-all"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForQuestionAnswering.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = MPNetTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

('onnx_models/haddadalwi/multi-qa-mpnet-base-dot-v1-finetuned-squad2-all/tokenizer_config.json',
 'onnx_models/haddadalwi/multi-qa-mpnet-base-dot-v1-finetuned-squad2-all/special_tokens_map.json',
 'onnx_models/haddadalwi/multi-qa-mpnet-base-dot-v1-finetuned-squad2-all/vocab.txt',
 'onnx_models/haddadalwi/multi-qa-mpnet-base-dot-v1-finetuned-squad2-all/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 425888
-rw-r--r-- 1 root root       647 Jun 15 23:43 config.json
-rw-r--r-- 1 root root 435859895 Jun 15 23:43 model.onnx
-rw-r--r-- 1 root root       964 Jun 15 23:43 special_tokens_map.json
-rw-r--r-- 1 root root      1475 Jun 15 23:43 tokenizer_config.json
-rw-r--r-- 1 root root    231536 Jun 15 23:43 vocab.txt


We have to move additional model assets (tokenizer vocabulary and configs) into a separate folder, so that Spark NLP can load it properly.

In [4]:
!mkdir -p {EXPORT_PATH}/assets && mv -t {EXPORT_PATH}/assets {EXPORT_PATH}/*.json {EXPORT_PATH}/*.txt

In [5]:
!ls -l {EXPORT_PATH}/assets

total 240
-rw-r--r-- 1 root root    647 Jun 15 23:43 config.json
-rw-r--r-- 1 root root    964 Jun 15 23:43 special_tokens_map.json
-rw-r--r-- 1 root root   1475 Jun 15 23:43 tokenizer_config.json
-rw-r--r-- 1 root root 231536 Jun 15 23:43 vocab.txt


## Import and Save MPNet in Spark NLP

- **Install and set up Spark NLP in Google Colab**
  - This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 32.2 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `MPNetForQuestionAnswering` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `MPNetForQuestionAnswering` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [8]:
from sparknlp.annotator import MPNetForQuestionAnswering

question_answering = (
    MPNetForQuestionAnswering.loadSavedModel(f"{EXPORT_PATH}", spark)
    .setInputCols("document_question", "document_context")
    .setOutputCol("answer")
)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
question_answering.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX MPNet model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {MODEL_NAME}_spark_nlp

total 425724
drwxr-xr-x 3 root root      4096 Jun 15 23:45 fields
drwxr-xr-x 2 root root      4096 Jun 15 23:45 metadata
-rw-r--r-- 1 root root 435926539 Jun 15 23:45 mpnet_question_answering_onnx


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny MPNet model 😊

In [18]:
from sparknlp.base import MultiDocumentAssembler
from sparknlp.annotator import MPNetForQuestionAnswering
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

question_answering = MPNetForQuestionAnswering.load(f"{MODEL_NAME}_spark_nlp") \
    .setInputCols(["document_question", "document_context"]) \
    .setOutputCol("answer") \
    .setCaseSensitive(False)

pipeline = Pipeline().setStages([
    document_assembler,
    question_answering
])

data = [
    ("What is the boiling point of water?", "Water boils at 100 degrees Celsius."),
    ("Who is the president of the USA?", "The president of the USA is Joe Biden."),
    ("What color is the sky?", "The sky is blue on a clear day."),
]
df = spark.createDataFrame(data, ["question", "context"])

result = pipeline.fit(df).transform(df)
result.select("question", "answer.result").show(truncate=False)

+-----------------------------------+---------------------+
|question                           |result               |
+-----------------------------------+---------------------+
|What is the boiling point of water?|[100 degrees Celsius]|
|Who is the president of the USA?   |[Joe Biden]          |
|What color is the sky?             |[blue]               |
+-----------------------------------+---------------------+



That's it! You can now go wild and use hundreds of MPNet models from HuggingFace 🤗 in Spark NLP 🚀
